In [ ]:
!pip install torch_geometric_temporal
!pip install -U torch-geometric  # or pip install torch-geometric --upgrade
!sed -i 's/from torch_geometric.utils.to_dense_adj import to_dense_adj/from torch_geometric.utils.dense import to_dense_adj/' /usr/local/lib/python3.11/dist-packages/torch_geometric_temporal/nn/attention/tsagcn.py
!sed -i 's/from torch_geometric.utils.dense import to_dense_adj/from torch_geometric.utils import to_dense_adj/' /usr/local/lib/python3.11/dist-packages/torch_geometric_temporal/nn/attention/tsagcn.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 24.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [ ]:
!pip install torch_geometric

In [ ]:
!pip install pydgn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.4/67.4 MB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade torch_geometric_temporal

In [ ]:
!pip install --upgrade torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric


In [ ]:


class MontevideoBusDatasetLoader(object):
    """A dataset of inflow passenger at bus stop level from Montevideo city.
    This dataset comprises hourly inflow passenger data at bus stop level for 11 bus lines during
    October 2020 from Montevideo city (Uruguay). The bus lines selected are the ones that carry
    people to the center of the city and they load more than 25% of the total daily inflow traffic.
    Vertices are bus stops, edges are links between bus stops when a bus line connects them and the
    weight represent the road distance. The target is the passenger inflow. This is a curated
    dataset made from different data sources of the Metropolitan Transportation System (STM) of
    Montevideo. These datasets are freely available to anyone in the National Catalog of Open Data
    from the government of Uruguay (https://catalogodatos.gub.uy/).
    """

    def __init__(self):
        self._read_web_data()

    def _read_web_data(self):
        url = "https://raw.githubusercontent.com/benedekrozemberczki/pytorch_geometric_temporal/master/dataset/montevideo_bus.json"
        self._dataset = json.loads(urllib.request.urlopen(url).read())

    def _get_node_ids(self):
        return [node.get('bus_stop') for node in self._dataset["nodes"]]

    def _get_edges(self):
        node_ids = self._get_node_ids()
        node_id_map = dict(zip(node_ids, range(len(node_ids))))
        self._edges = np.array(
            [(node_id_map[d["source"]], node_id_map[d["target"]]) for d in self._dataset["links"]]
        ).T

    def _get_edge_weights(self):
        self._edge_weights = np.array([(d["weight"]) for d in self._dataset["links"]]).T

    def _get_features(self, feature_vars: List[str] = ["y"]):
        features = []
        for node in self._dataset["nodes"]:
            X = node.get("X")
            for feature_var in feature_vars:
                features.append(np.array(X.get(feature_var)))
        stacked_features = np.stack(features).T
        standardized_features = (
            stacked_features - np.mean(stacked_features, axis=0)
        ) / np.std(stacked_features, axis=0)
        self.features = [
            standardized_features[i : i + self.lags, :].T
            for i in range(len(standardized_features) - self.lags)
        ]

    def _get_targets(self, target_var: str = "y"):
        targets = []
        for node in self._dataset["nodes"]:
            y = node.get(target_var)
            targets.append(np.array(y))
        stacked_targets = np.stack(targets).T
        standardized_targets = (
            stacked_targets - np.mean(stacked_targets, axis=0)
        ) / np.std(stacked_targets, axis=0)
        self.targets = [
            standardized_targets[i + self.lags, :].T
            for i in range(len(standardized_targets) - self.lags)
        ]
    def get_dataset(
        self, lags: int = 4, target_var: str = "y", feature_vars: List[str] = ["y"]
    ) -> StaticGraphTemporalSignal:
        """Returning the MontevideoBus passenger inflow data iterator.

        Parameters
        ----------
        lags : int, optional
            The number of time lags, by default 4.
        target_var : str, optional
            Target variable name, by default "y".
        feature_vars : List[str], optional
            List of feature variables, by default ["y"].

        Returns
        -------
        StaticGraphTemporalSignal
            The MontevideoBus dataset.
        """
        self.lags = lags
        self._get_edges()
        self._get_edge_weights()
        self._get_features(feature_vars)
        self._get_targets(target_var)
        dataset = StaticGraphTemporalSignal(
            self._edges, self._edge_weights, self.features, self.targets
        )
        return dataset

In [ ]:

class A3TGCN(nn.Module):
    """
    A3T-GCN model for spatio-temporal forecasting.
    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(A3TGCN, self).__init__()
        self.gcn = nn.Linear(input_dim, hidden_dim)  # Placeholder GCN layer
        self.attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=2)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.gcn(x)
        x, _ = self.attention(x, x, x)
        x = self.fc(x)
        return x

if __name__ == "__main__":
    input_dim = 16  # Placeholder value, replace with actual dataset feature size
    model = A3TGCN(input_dim=input_dim, hidden_dim=64, output_dim=1)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    print("A3T-GCN model initialized.")


A3T-GCN model initialized.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import json
import urllib.request
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split
from typing import List
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

class A3TGCN(nn.Module):
    """
    A3T-GCN model for spatio-temporal forecasting.
    """
    def __init__(self, input_dim, hidden_dim, num_nodes):
        super(A3TGCN, self).__init__()
        self.gcn = nn.Linear(input_dim, hidden_dim)  # Placeholder GCN layer
        self.attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=2, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)  # Ensure output matches num_nodes

    def forward(self, x):
        batch_size, num_nodes, num_features = x.shape

        # Reshape for Linear Layer
        x = x.view(-1, num_features)  # (batch_size * num_nodes, input_dim)

        x = self.gcn(x)  # (batch_size * num_nodes, hidden_dim)

        # Reshape back to (batch_size, num_nodes, hidden_dim)
        x = x.view(batch_size, num_nodes, -1)

        x, _ = self.attention(x, x, x)  # (batch_size, num_nodes, hidden_dim)

        x = self.fc(x)  # (batch_size, num_nodes, 1)

        return x.squeeze(-1)  # Final shape: (batch_size, num_nodes)

class MontevideoBusDatasetLoader:
    def __init__(self):
        self._read_web_data()

    def _read_web_data(self):
        url = "https://raw.githubusercontent.com/benedekrozemberczki/pytorch_geometric_temporal/master/dataset/montevideo_bus.json"
        self._dataset = json.loads(urllib.request.urlopen(url).read())

    def _get_node_ids(self):
        return [node.get('bus_stop') for node in self._dataset["nodes"]]

    def _get_edges(self):
        node_ids = self._get_node_ids()
        node_id_map = dict(zip(node_ids, range(len(node_ids))))
        self._edges = np.array(
            [(node_id_map[d["source"]], node_id_map[d["target"]]) for d in self._dataset["links"]]
        ).T

    def _get_edge_weights(self):
        self._edge_weights = np.array([(d["weight"]) for d in self._dataset["links"]]).T

    def _get_features(self, feature_vars: List[str] = ["y"]):
        features = []
        for node in self._dataset["nodes"]:
            X = node.get("X")
            for feature_var in feature_vars:
                features.append(np.array(X.get(feature_var)))
        stacked_features = np.stack(features).T
        standardized_features = (
            stacked_features - np.mean(stacked_features, axis=0)
        ) / np.std(stacked_features, axis=0)
        self.features = [
            standardized_features[i : i + self.lags, :].T
            for i in range(len(stacked_features) - self.lags)
        ]

    def _get_targets(self, target_var: str = "y"):
        targets = []
        for node in self._dataset["nodes"]:
            y = node.get(target_var)
            targets.append(np.array(y))
        stacked_targets = np.stack(targets).T
        standardized_targets = (
            stacked_targets - np.mean(stacked_targets, axis=0)
        ) / np.std(stacked_targets, axis=0)
        self.targets = [
            standardized_targets[i + self.lags, :].T
            for i in range(len(stacked_targets) - self.lags)
        ]

    def get_dataset(self, lags: int = 4, target_var: str = "y", feature_vars: List[str] = ["y"]):
        self.lags = lags
        self._get_edges()
        self._get_edge_weights()
        self._get_features(feature_vars)
        self._get_targets(target_var)
        dataset = StaticGraphTemporalSignal(
            self._edges, self._edge_weights, self.features, self.targets
        )
        return dataset

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch.float())
            loss = criterion(y_pred, y_batch.float())
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(train_loader)}")

def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch.float())
            loss = criterion(y_pred, y_batch.float())
            test_loss += loss.item()
    print(f"Test Loss: {test_loss / len(test_loader)}")

if __name__ == "__main__":
    dataset_loader = MontevideoBusDatasetLoader()
    dataset = dataset_loader.get_dataset()

    #input_dim = len(dataset.features[0])
    print("Feature Shape:", np.array(dataset.features).shape)
    input_dim = dataset.features[0].shape[-1]  # Ensure correct input dimension
    num_nodes = dataset.targets[0].shape[0]  # Ensure correct output size
    print("Input Dimension:", input_dim)
    print("Number of Nodes:", num_nodes)
    model = A3TGCN(input_dim=input_dim, hidden_dim=64, num_nodes=num_nodes)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    # Convert dataset into tensors
    X_tensor = torch.tensor(np.array(dataset.features), dtype=torch.float32)
    y_tensor = torch.tensor(np.array(dataset.targets), dtype=torch.float32)

    # Split into train and test sets
    dataset_size = len(X_tensor)
    train_size = int(0.8 * dataset_size)
    test_size = dataset_size - train_size
    train_dataset, test_dataset = random_split(TensorDataset(X_tensor, y_tensor), [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Train and test the model
    train_model(model, train_loader, criterion, optimizer, epochs=10)
    test_model(model, test_loader, criterion)

    print("A3T-GCN model training and testing completed.")


Feature Shape: (740, 675, 4)
Input Dimension: 4
Number of Nodes: 675
Epoch 1, Loss: 0.9566035615770441
Epoch 2, Loss: 0.913731386787013
Epoch 3, Loss: 0.9207206337075484
Epoch 4, Loss: 0.913191911421324
Epoch 5, Loss: 0.9087845683097839
Epoch 6, Loss: 0.9121853458253961
Epoch 7, Loss: 0.921468624943181
Epoch 8, Loss: 0.9077176859504298
Epoch 9, Loss: 0.9090192819896498
Epoch 10, Loss: 0.914704325952028
Test Loss: 0.9220641016960144
A3T-GCN model training and testing completed.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import json
import urllib.request
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from typing import List
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
from torch_geometric.nn import ChebConv
import torch.nn.functional as F

##########################################
#        MODEL DEFINITION                #
##########################################

class TemporalGCN(torch.nn.Module):
    def __init__(self, in_channels: int, out_channels: int, improved: bool = False,
                 cached: bool = False, bias: bool = True):
        super(TemporalGCN, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved
        self.cached = cached
        self.bias = bias

        self.conv_z = ChebConv(in_channels, out_channels, K=2, bias=bias)
        self.conv_r = ChebConv(in_channels, out_channels, K=2, bias=bias)
        self.conv_h = ChebConv(in_channels, out_channels, K=2, bias=bias)
        self.linear_z = nn.Linear(2 * out_channels, out_channels)
        self.linear_r = nn.Linear(2 * out_channels, out_channels)
        self.linear_h = nn.Linear(2 * out_channels, out_channels)

    def _set_hidden_state(self, X, H):
        if H is None:
            H = torch.zeros(X.shape[0], X.shape[1], self.out_channels).to(X.device)
        return H

    def _calculate_update_gate(self, X, edge_index, edge_weight, H):
        Z = torch.cat([self.conv_z(X, edge_index, edge_weight), H], dim=2)
        Z = self.linear_z(Z.reshape(-1, 2 * self.out_channels)).reshape(X.shape[0], X.shape[1], self.out_channels)
        Z = torch.sigmoid(Z)
        return Z

    def _calculate_reset_gate(self, X, edge_index, edge_weight, H):
        R = torch.cat([self.conv_r(X, edge_index, edge_weight), H], dim=2)
        R = self.linear_r(R.reshape(-1, 2 * self.out_channels)).reshape(X.shape[0], X.shape[1], self.out_channels)
        R = torch.sigmoid(R)
        return R

    def _calculate_candidate_state(self, X, edge_index, edge_weight, H, R):
        H_tilde = self.conv_h(X, edge_index, edge_weight)
        H_tilde = torch.cat([H_tilde, H * R], dim=2)
        H_tilde = self.linear_h(H_tilde.reshape(-1, 2 * self.out_channels)).reshape(X.shape[0], X.shape[1], self.out_channels)
        H_tilde = torch.tanh(H_tilde)
        return H_tilde

    def forward(self, X: torch.FloatTensor, edge_index: torch.LongTensor,
                edge_weight: torch.FloatTensor, H: torch.FloatTensor = None) -> torch.FloatTensor:
        H = self._set_hidden_state(X, H)
        Z = self._calculate_update_gate(X, edge_index, edge_weight, H)
        R = self._calculate_reset_gate(X, edge_index, edge_weight, H)
        H_tilde = self._calculate_candidate_state(X, edge_index, edge_weight, H, R)
        H = Z * H + (1 - Z) * H_tilde
        return H

class TGCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_nodes, edge_index, edge_weight):
        super(TGCNModel, self).__init__()
        self.tgcn = TemporalGCN(input_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, 1)
        self.num_nodes = num_nodes
        self.edge_index = edge_index
        self.edge_weight = edge_weight
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        """
        x: Tensor of shape (batch, T, N, F)
        We'll iterate over the batch dimension.
        Returns: prediction (batch, N) from the last time step.
        """
        batch_size = x.shape[0]
        outputs = []
        for i in range(batch_size):
            xi = x[i]  # shape: (T, N, F)
            if h is None:
                hi = torch.zeros(xi.shape[0], self.num_nodes, self.hidden_dim, device=xi.device)
            else:
                hi = h[i]
            out = self.tgcn(xi, self.edge_index, self.edge_weight, hi)  # out: (T, N, hidden_dim)
            out = self.fc(out)  # out: (T, N, 1)
            outputs.append(out)
        outputs = torch.stack(outputs, dim=0)  # shape: (batch, T, N, 1)
        # Return the prediction at the last time step for each node
        return outputs[:, -1, :, 0], outputs

##########################################
#        DATA LOADING                    #
##########################################

class MontevideoBusDatasetLoader:
    def __init__(self):
        self._read_web_data()

    def _read_web_data(self):
        url = "https://raw.githubusercontent.com/benedekrozemberczki/pytorch_geometric_temporal/master/dataset/montevideo_bus.json"
        self._dataset = json.loads(urllib.request.urlopen(url).read())

    def _get_node_ids(self):
        return [node.get('bus_stop') for node in self._dataset["nodes"]]

    def _get_edges(self):
        node_ids = self._get_node_ids()
        node_id_map = dict(zip(node_ids, range(len(node_ids))))
        edges = np.array(
            [(node_id_map[d["source"]], node_id_map[d["target"]]) for d in self._dataset["links"]]
        ).T
        return torch.tensor(edges, dtype=torch.long)

    def _get_edge_weights(self):
        self._edge_weights = np.array([d["weight"] for d in self._dataset["links"]])
        return torch.tensor(self._edge_weights, dtype=torch.float)

    def _get_features(self, feature_vars: List[str] = ["y"]):
        features = []
        for node in self._dataset["nodes"]:
            X = node.get("X")
            for feature_var in feature_vars:
                features.append(np.array(X.get(feature_var)))
        # stacked_features shape: (T, num_nodes)
        stacked_features = np.stack(features).T
        standardized_features = (
            stacked_features - np.mean(stacked_features, axis=0)
        ) / np.std(stacked_features, axis=0)
        # For each sliding window, each sample is of shape (num_nodes, lags)
        self.features = [
            standardized_features[i: i + self.lags, :].T
            for i in range(len(stacked_features) - self.lags)
        ]

    def _get_targets(self, target_var: str = "y"):
        targets = []
        for node in self._dataset["nodes"]:
            y = node.get(target_var)
            targets.append(np.array(y))
        stacked_targets = np.stack(targets).T
        standardized_targets = (
            stacked_targets - np.mean(stacked_targets, axis=0)
        ) / np.std(stacked_targets, axis=0)
        # For each sliding window, the target is the next time step for each node.
        self.targets = [
            standardized_targets[i + self.lags, :]
            for i in range(len(stacked_targets) - self.lags)
        ]

    def get_dataset(self, lags: int = 4, target_var: str = "y", feature_vars: List[str] = ["y"]):
        self.lags = lags
        edges = self._get_edges()
        edge_weights = self._get_edge_weights()
        self._get_features(feature_vars)
        self._get_targets(target_var)
        dataset = StaticGraphTemporalSignal(
            edges, edge_weights, self.features, self.targets
        )
        return dataset

##########################################
#        TRAINING & TESTING              #
##########################################

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            # X_batch shape: (batch, T, N, F)
            y_pred, _ = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(train_loader):.4f}")

def test_model(model, test_loader):
    model.eval()
    predictions, targets = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred, _ = model(X_batch)
            predictions.append(y_pred)
            targets.append(y_batch)
    predictions = torch.cat(predictions, dim=0)
    targets = torch.cat(targets, dim=0)
    return predictions, targets

##########################################
#            MAIN SCRIPT                 #
##########################################

# Load dataset
dataset_loader = MontevideoBusDatasetLoader()
dataset = dataset_loader.get_dataset(lags=4)

# The dataset provides a list of features (one per time window).
num_time_steps = len(dataset.features)
train_size = int(0.8 * num_time_steps)
test_size = num_time_steps - train_size

# --- Process features ---
# Each element in dataset.features has shape: (num_nodes, lags)
# We want to convert each sample to shape: (lags, num_nodes, 1)
# First, stack the list into a NumPy array, then transpose and add the feature dimension.
train_features_np = np.stack(dataset.features[:train_size], axis=0)  # shape: (train_samples, num_nodes, lags)
train_features_np = np.transpose(train_features_np, (0, 2, 1))         # shape: (train_samples, lags, num_nodes)
train_features_np = np.expand_dims(train_features_np, -1)               # shape: (train_samples, lags, num_nodes, 1)

test_features_np = np.stack(dataset.features[train_size:], axis=0)      # shape: (test_samples, num_nodes, lags)
test_features_np = np.transpose(test_features_np, (0, 2, 1))             # shape: (test_samples, lags, num_nodes)
test_features_np = np.expand_dims(test_features_np, -1)                  # shape: (test_samples, lags, num_nodes, 1)

# --- Process targets ---
# Each element in dataset.targets has shape: (num_nodes,)
train_targets_np = np.stack(dataset.targets[:train_size], axis=0)       # shape: (train_samples, num_nodes)
test_targets_np = np.stack(dataset.targets[train_size:], axis=0)          # shape: (test_samples, num_nodes)

# Convert to torch tensors (using np.stack avoids the warning)
train_features_tensor = torch.tensor(train_features_np, dtype=torch.float)
train_targets_tensor = torch.tensor(train_targets_np, dtype=torch.float)
test_features_tensor = torch.tensor(test_features_np, dtype=torch.float)
test_targets_tensor = torch.tensor(test_targets_np, dtype=torch.float)

# Create DataLoader – here we use batch_size=1 so that each sample (a sliding window) is processed individually.
batch_size = 1
train_loader = DataLoader(TensorDataset(train_features_tensor, train_targets_tensor), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(test_features_tensor, test_targets_tensor), batch_size=batch_size, shuffle=False)

# Derive model dimensions:
# - The feature dimension (F) is the last dimension (which is 1).
# - The number of nodes (N) is the 3rd dimension (index 2) of our data.
input_dim = train_features_tensor.shape[-1]  # should be 1
num_nodes = train_features_tensor.shape[2]
hidden_dim = 64
edge_index = dataset.edge_index
edge_weight = dataset.edge_weight

# Initialize the model, criterion, and optimizer.
model = TGCNModel(input_dim, hidden_dim, num_nodes, edge_index, edge_weight)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model.
train_model(model, train_loader, criterion, optimizer, epochs=10)

# Test the model.
predictions, targets = test_model(model, test_loader)

# Compute the test loss.
test_loss = criterion(predictions, targets)
print("Test Loss:", test_loss.item())

print("Test Predictions:", predictions)
print("Test Targets:", targets)


RuntimeError: index 32 is out of bounds for dimension 0 with size 32

In [ ]:
# Working with the 2017 CitiBike dataset
# Step 1: Import necessary libraries
import os
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_scipy_sparse_matrix
from tqdm import tqdm
from os.path import join, isfile, exists
import zipfile
import requests
from io import BytesIO
import numpy as np


# Step 2: Define the CitiBikeDatasetInterface class
class CitiBikeDatasetInterface:
    """
    CitiBike dataset interface for processing and accessing 2017 CitiBike data.
    """

    def __init__(self, root, url, num_timesteps_in=1, num_timesteps_out=1):
        self.root = root
        self.url = url
        self.num_timesteps_in = num_timesteps_in
        self.num_timesteps_out = num_timesteps_out
        self.combined_csv_path = join(root, "2017_combined_data.csv")
        self.data_path = join(root, "2017-citibike-tripdata.zip")

        # Check if processed dataset exists; if not, download and process it
        if not exists(self.combined_csv_path):  # Check if combined CSV exists
            if not exists(self.data_path):  # Check if the zip file exists
                self.download_and_extract()
            self.combine_csv_files()


        self.load_data()

    def download_and_extract(self):
        # Download the dataset from the provided URL
        print("Downloading dataset...")
        response = requests.get(self.url, stream=True) # Download in chunks
        total_size_in_bytes= int(response.headers.get('content-length', 0))
        block_size = 1024 #1 Kibibyte
        progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
        with open(self.data_path, 'wb') as file:
            for data in response.iter_content(block_size):
                progress_bar.update(len(data))
                file.write(data)
        progress_bar.close()
        if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
            print("ERROR, something went wrong")

        print("Dataset downloaded.")

        print("Extracting dataset...")
        with zipfile.ZipFile(self.data_path, 'r') as zip_ref:
            zip_ref.extractall(self.root)
        print("Dataset extracted.")


    def combine_csv_files(self):
        print("Combining monthly data into a single CSV...")
        all_files = []
        # Updated to directly access the zip file
        with zipfile.ZipFile(self.data_path, 'r') as zip_ref:
            for file_info in zip_ref.infolist():
                if file_info.filename.endswith(".csv"):
                    file_info.filename = os.path.basename(file_info.filename)  # Remove path information
                    zip_ref.extract(file_info, self.root)
                    all_files.append(join(self.root, file_info.filename))

        combined_df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
        combined_df.to_csv(self.combined_csv_path, index=False)
        print(f"Combined CSV saved to {self.combined_csv_path}")

    def load_data(self):
        print("Loading data...")
        df = pd.read_csv(self.combined_csv_path)

        # Example: Processing data into graph format
        # Each node could represent a bike station, and edges represent trips between stations
        self.node_features = torch.tensor(df.iloc[:, :5].values, dtype=torch.float)  # Example feature columns
        self.targets = torch.tensor(df.iloc[:, 5].values, dtype=torch.float).unsqueeze(-1)  # Example target column

        # Create a dummy edge index (in real scenarios, use actual graph structure)
        num_nodes = self.node_features.shape[0]
        self.edge_index = torch.tensor([[i, (i + 1) % num_nodes] for i in range(num_nodes)]).t().contiguous()

    @property
    def dim_node_features(self):
        return self.node_features.shape[1]

    @property
    def dim_edge_features(self):
        return 0

    @property
    def dim_target(self):
        return self.targets.shape[1]

    def __len__(self):
        return len(self.node_features)

    def __getitem__(self, idx):
        return Data(
            x=self.node_features[idx],
            edge_index=self.edge_index,
            y=self.targets[idx]
        )


# Step 3: Test the class in the same notebook
# Define the root directory and dataset URL
root = './data'
url = 'https://s3.amazonaws.com/tripdata/2017-citibike-tripdata.zip'

# Create an instance of the class
dataset = CitiBikeDatasetInterface(root=root, url=url)

# Print dataset details
print(f"Number of time steps: {len(dataset)}")
print(f"Node feature dimension: {dataset.dim_node_features}")
print(f"Edge feature dimension: {dataset.dim_edge_features}")
print(f"Target dimension: {dataset.dim_target}")

# Access and inspect a single time step
time_index = 0
data = dataset[time_index]

print("\nExample Data Object:")
print(f"Node Features (x):\n{data.x}")
print(f"Edge Index:\n{data.edge_index}")
print(f"Target (y):\n{data.y}")

100%|██████████| 557M/557M [00:16<00:00, 33.7MiB/s]


Dataset downloaded.
Extracting dataset...


KeyboardInterrupt: 

In [ ]:
import os
import gc
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
from tqdm import tqdm
import zipfile
import requests
import numpy as np

# CitiBike Dataset Interface
class CitiBikeDatasetInterface:
    def __init__(self, root, url, num_timesteps_in=4, num_timesteps_out=1):
        self.root = root
        self.url = url
        self.num_timesteps_in = num_timesteps_in
        self.num_timesteps_out = num_timesteps_out
        self.combined_csv_path = os.path.join(root, "2017_combined_data.csv")
        self.data_path = os.path.join(root, "2017-citibike-tripdata.zip")

        if not os.path.exists(self.combined_csv_path):
            if not os.path.exists(self.data_path):
                self.download_and_extract()
            self.combine_csv_files()

    def download_and_extract(self):
        print("Downloading dataset...")
        response = requests.get(self.url, stream=True)
        with open(self.data_path, 'wb') as file:
            for data in response.iter_content(1024):
                file.write(data)
        print("Dataset downloaded.")

        print("Extracting dataset...")
        with zipfile.ZipFile(self.data_path, 'r') as zip_ref:
            zip_ref.extractall(self.root)
        print("Dataset extracted.")

    def combine_csv_files(self, chunksize=100000):
        print("Combining monthly data into a single CSV...")
        all_files = []
        with zipfile.ZipFile(self.data_path, 'r') as zip_ref:
            for file_info in zip_ref.infolist():
                if file_info.filename.endswith(".csv"):
                    file_info.filename = os.path.basename(file_info.filename)
                    zip_ref.extract(file_info, self.root)
                    all_files.append(os.path.join(self.root, file_info.filename))

        with open(self.combined_csv_path, 'w') as f_out:
            for i, f in enumerate(all_files):
                for chunk in pd.read_csv(f, chunksize=chunksize):
                    chunk.to_csv(f_out, mode='a', index=False, header=(i == 0))

        print(f"Combined CSV saved to {self.combined_csv_path}")

    def load_and_process_data(self, chunksize=100000):
        print("Processing data in chunks...")
        node_features = []
        targets = []

        for chunk in pd.read_csv(self.combined_csv_path, chunksize=chunksize):
            chunk = chunk.sort_values(by=["starttime"])
            station_counts = chunk.groupby("start station id").size().reset_index(name="trip_count")

            node_features.append(torch.tensor(station_counts["trip_count"].values, dtype=torch.float32).unsqueeze(-1))
            targets.append(node_features[-1].clone())

        self.node_features = torch.cat(node_features, dim=0)
        self.targets = torch.cat(targets, dim=0)

        num_nodes = self.node_features.shape[0]
        self.edge_index = torch.tensor([[i, (i + 1) % num_nodes] for i in range(num_nodes)]).t().contiguous()

        # Free memory
        del node_features, targets, chunk
        gc.collect()

    def get_dataset(self):
        self.load_and_process_data()
        dataset = CitiBikeDataset(self.node_features, self.targets, self.num_timesteps_in)
        return dataset, self.edge_index


# Memory-Efficient Dataset Class
class CitiBikeDataset(Dataset):
    def __init__(self, node_features, targets, num_timesteps_in):
        self.node_features = node_features
        self.targets = targets
        self.num_timesteps_in = num_timesteps_in

    def __len__(self):
        return len(self.node_features) - self.num_timesteps_in

    def __getitem__(self, idx):
        x = self.node_features[idx: idx + self.num_timesteps_in]
        y = self.targets[idx + self.num_timesteps_in]
        return x, y


# A3T-GCN Model
class A3TGCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_nodes):
        super(A3TGCN, self).__init__()
        self.gcn = nn.Linear(input_dim, hidden_dim)
        self.attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=2, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        batch_size, num_nodes, num_features = x.shape
        x = x.view(-1, num_features)
        x = self.gcn(x)
        x = x.view(batch_size, num_nodes, -1)
        x, _ = self.attention(x, x, x)
        x = self.fc(x)
        return x.squeeze(-1)


# Training and Testing
if __name__ == "__main__":
    dataset_loader = CitiBikeDatasetInterface(root='./data', url='https://s3.amazonaws.com/tripdata/2017-citibike-tripdata.zip')
    dataset, edge_index = dataset_loader.get_dataset()

    # Set up DataLoader
    batch_size = 32
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    # Model Setup
    input_dim = dataset.node_features.shape[-1]
    num_nodes = dataset.node_features.shape[0]
    model = A3TGCN(input_dim=input_dim, hidden_dim=64, num_nodes=num_nodes)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    # Training Loop
    for epoch in range(10):
        model.train()
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch.float())
            loss = criterion(y_pred, y_batch.float())
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(train_loader)}")

    # Testing Loop
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch.float())
            loss = criterion(y_pred, y_batch.float())
            test_loss += loss.item()
    print(f"Test Loss: {test_loss / len(test_loader)}")

    # Free memory
    del dataset, train_dataset, test_dataset, train_loader, test_loader
    gc.collect()
    torch.cuda.empty_cache()  # If using GPU


Processing data in chunks...


<ipython-input-1-37ad7ce11fec>:124: DtypeWarning: Columns (1,2,4,8,12,16,17,19,23,27) have mixed types.Specify dtype option on import or set low_memory=False.
  dataset, edge_index = dataset_loader.get_dataset()
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 16138.324297542771


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([11, 1])) that is different to the input size (torch.Size([11, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 2, Loss: 15690.691955885406
Epoch 3, Loss: 15263.56602483344
Epoch 4, Loss: 15098.543139839838
Epoch 5, Loss: 15036.339172575947
Epoch 6, Loss: 15036.215854658067
Epoch 7, Loss: 14994.234946752675
Epoch 8, Loss: 15033.221074439924
Epoch 9, Loss: 14989.938582470193
Epoch 10, Loss: 14966.073861364703


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([27, 1])) that is different to the input size (torch.Size([27, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Loss: 14880.922015359592


In [ ]:
import os
import gc
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
from tqdm import tqdm
import zipfile
import requests
import numpy as np
from torch_geometric.nn import ChebConv
import torch.nn.functional as F

##########################################
#      CitiBike Dataset Interface        #
##########################################

class CitiBikeDatasetInterface:
    def __init__(self, root, url, num_timesteps_in=4, num_timesteps_out=1):
        self.root = root
        self.url = url
        self.num_timesteps_in = num_timesteps_in
        self.num_timesteps_out = num_timesteps_out
        self.combined_csv_path = os.path.join(root, "2017_combined_data.csv")
        self.data_path = os.path.join(root, "2017-citibike-tripdata.zip")

        if not os.path.exists(self.combined_csv_path):
            if not os.path.exists(self.data_path):
                self.download_and_extract()
            self.combine_csv_files()

    def download_and_extract(self):
        print("Downloading dataset...")
        response = requests.get(self.url, stream=True)
        with open(self.data_path, 'wb') as file:
            for data in response.iter_content(1024):
                file.write(data)
        print("Dataset downloaded.")

        print("Extracting dataset...")
        with zipfile.ZipFile(self.data_path, 'r') as zip_ref:
            zip_ref.extractall(self.root)
        print("Dataset extracted.")

    def combine_csv_files(self, chunksize=100000):
        print("Combining monthly data into a single CSV...")
        all_files = []
        with zipfile.ZipFile(self.data_path, 'r') as zip_ref:
            for file_info in zip_ref.infolist():
                if file_info.filename.endswith(".csv"):
                    file_info.filename = os.path.basename(file_info.filename)
                    zip_ref.extract(file_info, self.root)
                    all_files.append(os.path.join(self.root, file_info.filename))

        with open(self.combined_csv_path, 'w') as f_out:
            for i, f in enumerate(all_files):
                for chunk in pd.read_csv(f, chunksize=chunksize):
                    chunk.to_csv(f_out, mode='a', index=False, header=(i == 0))
        print(f"Combined CSV saved to {self.combined_csv_path}")

    def load_and_process_data(self, chunksize=100000):
        print("Computing union of station IDs...")
        # First pass: compute the union of all station IDs.
        station_id_set = set()
        for chunk in pd.read_csv(self.combined_csv_path, chunksize=chunksize, usecols=["start station id"]):
            # Convert station IDs to string to ensure consistent type.
            chunk["start station id"] = chunk["start station id"].astype(str)
            station_id_set.update(chunk["start station id"].unique())
        station_ids = sorted(list(station_id_set))
        self.station_ids = station_ids
        num_stations = len(station_ids)
        print(f"Found {num_stations} unique station IDs.")

        # Create a mapping from station id (as string) to index.
        station_id_map = {station_id: idx for idx, station_id in enumerate(station_ids)}

        print("Processing data in chunks...")
        node_features = []
        targets = []
        for chunk in pd.read_csv(self.combined_csv_path, chunksize=chunksize):
            # Ensure station IDs are strings.
            chunk["start station id"] = chunk["start station id"].astype(str)
            # Sort by time so that our sliding window is in order.
            chunk = chunk.sort_values(by=["starttime"])
            # Group by station id to get trip counts.
            station_counts = chunk.groupby("start station id").size().reset_index(name="trip_count")
            # Create a numpy array of zeros with length = num_stations.
            arr = np.zeros(num_stations, dtype=np.float32)
            # Fill in the counts for the stations present in this chunk.
            for _, row in station_counts.iterrows():
                station_id = row["start station id"]
                count = row["trip_count"]
                # Use the mapping to determine the correct index.
                idx = station_id_map[station_id]
                arr[idx] = count
            # Convert to tensor and shape becomes (num_stations, 1)
            ts = torch.tensor(arr, dtype=torch.float32).unsqueeze(-1)
            node_features.append(ts)
            targets.append(ts.clone())
        # Stack along a new (time) dimension: shape = (T, num_stations, 1)
        self.node_features = torch.stack(node_features, dim=0)
        self.targets = torch.stack(targets, dim=0)

        # Create a simple ring (cycle) over nodes.
        self.edge_index = torch.tensor(
            [[i, (i + 1) % num_stations] for i in range(num_stations)]
        ).t().contiguous()


    def get_dataset(self):
        self.load_and_process_data()
        dataset = CitiBikeDataset(self.node_features, self.targets, self.num_timesteps_in)
        return dataset, self.edge_index

##########################################
#         CitiBike Dataset Class         #
##########################################

class CitiBikeDataset(Dataset):
    def __init__(self, node_features, targets, num_timesteps_in):
        """
        node_features: Tensor of shape (T, N, F)
        targets: Tensor of shape (T, N, F)
        """
        self.node_features = node_features
        self.targets = targets
        self.num_timesteps_in = num_timesteps_in

    def __len__(self):
        # Number of sliding windows available
        return self.node_features.shape[0] - self.num_timesteps_in

    def __getitem__(self, idx):
        # x: window of past time steps, shape: (num_timesteps_in, N, F)
        # y: next time step target, shape: (N, F)
        x = self.node_features[idx: idx + self.num_timesteps_in]
        y = self.targets[idx + self.num_timesteps_in]
        # Squeeze the last dimension of y so that it is (N)
        return x, y.squeeze(-1)

##########################################
#       TemporalGCN & TGCNModel          #
##########################################

class TemporalGCN(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, improved: bool = False,
                 cached: bool = False, bias: bool = True):
        super(TemporalGCN, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.conv_z = ChebConv(in_channels, out_channels, K=2, bias=bias)
        self.conv_r = ChebConv(in_channels, out_channels, K=2, bias=bias)
        self.conv_h = ChebConv(in_channels, out_channels, K=2, bias=bias)
        self.linear_z = nn.Linear(2 * out_channels, out_channels)
        self.linear_r = nn.Linear(2 * out_channels, out_channels)
        self.linear_h = nn.Linear(2 * out_channels, out_channels)

    def _set_hidden_state(self, X, H):
        if H is None:
            H = torch.zeros(X.shape[0], X.shape[1], self.out_channels, device=X.device)
        return H

    def _calculate_update_gate(self, X, edge_index, edge_weight, H):
        Z = torch.cat([self.conv_z(X, edge_index, edge_weight), H], dim=2)
        Z = self.linear_z(Z.reshape(-1, 2 * self.out_channels)).reshape(X.shape[0], X.shape[1], self.out_channels)
        Z = torch.sigmoid(Z)
        return Z

    def _calculate_reset_gate(self, X, edge_index, edge_weight, H):
        R = torch.cat([self.conv_r(X, edge_index, edge_weight), H], dim=2)
        R = self.linear_r(R.reshape(-1, 2 * self.out_channels)).reshape(X.shape[0], X.shape[1], self.out_channels)
        R = torch.sigmoid(R)
        return R

    def _calculate_candidate_state(self, X, edge_index, edge_weight, H, R):
        H_tilde = self.conv_h(X, edge_index, edge_weight)
        H_tilde = torch.cat([H_tilde, H * R], dim=2)
        H_tilde = self.linear_h(H_tilde.reshape(-1, 2 * self.out_channels)).reshape(X.shape[0], X.shape[1], self.out_channels)
        H_tilde = torch.tanh(H_tilde)
        return H_tilde

    def forward(self, X: torch.FloatTensor, edge_index: torch.LongTensor,
                edge_weight: torch.FloatTensor, H: torch.FloatTensor = None) -> torch.FloatTensor:
        H = self._set_hidden_state(X, H)
        Z = self._calculate_update_gate(X, edge_index, edge_weight, H)
        R = self._calculate_reset_gate(X, edge_index, edge_weight, H)
        H_tilde = self._calculate_candidate_state(X, edge_index, edge_weight, H, R)
        H = Z * H + (1 - Z) * H_tilde
        return H

class TGCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_nodes, edge_index, edge_weight):
        super(TGCNModel, self).__init__()
        self.tgcn = TemporalGCN(input_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, 1)
        self.num_nodes = num_nodes
        self.edge_index = edge_index
        self.edge_weight = edge_weight
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        """
        x: Tensor of shape (batch, T, N, F)
        Returns: predictions for the last time step, shape: (batch, N)
        """
        batch_size = x.shape[0]
        outputs = []
        for i in range(batch_size):
            xi = x[i]  # shape: (T, N, F)
            if h is None:
                hi = torch.zeros(xi.shape[0], self.num_nodes, self.hidden_dim, device=xi.device)
            else:
                hi = h[i]
            out = self.tgcn(xi, self.edge_index, self.edge_weight, hi)  # (T, N, hidden_dim)
            out = self.fc(out)  # (T, N, 1)
            outputs.append(out)
        outputs = torch.stack(outputs, dim=0)  # (batch, T, N, 1)
        return outputs[:, -1, :, 0], outputs  # return predictions from the last time step

##########################################
#         Training & Testing             #
##########################################

if __name__ == "__main__":
    # Setup paths and parameters
    root = './data'
    url = 'https://s3.amazonaws.com/tripdata/2017-citibike-tripdata.zip'
    num_timesteps_in = 4  # window length for input
    num_timesteps_out = 1

    # Get the dataset and edge_index from the CitiBike interface.
    dataset_interface = CitiBikeDatasetInterface(root, url, num_timesteps_in, num_timesteps_out)
    dataset_raw, edge_index = dataset_interface.get_dataset()

    # Here, node_features now has shape (T, N, F); determine number of nodes from dimension 1.
    num_nodes = dataset_interface.node_features.shape[1]

    # The CitiBikeDataset is a sliding-window over the time axis.
    dataset = dataset_raw

    # Split dataset into training and testing parts.
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    # Create DataLoaders (batch size 32)
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    # Model Setup:
    # - Input dimension F is the last dimension of node_features (should be 1).
    input_dim = dataset_interface.node_features.shape[-1]
    hidden_dim = 64
    # For edge_weight, we simply assign ones (one weight per edge).
    edge_weight = torch.ones(edge_index.shape[1], dtype=torch.float)

    model = TGCNModel(input_dim, hidden_dim, num_nodes, edge_index, edge_weight)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    # Training Loop
    for epoch in range(10):
        model.train()
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            # X_batch: shape (batch, num_timesteps_in, N, F)
            # y_batch: shape (batch, N)
            y_pred, _ = model(X_batch.float())
            loss = criterion(y_pred, y_batch.float())
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(train_loader):.4f}")

    # Testing Loop
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred, _ = model(X_batch.float())
            loss = criterion(y_pred, y_batch.float())
            test_loss += loss.item()
    print(f"Test Loss: {test_loss / len(test_loader):.4f}")

    # Free memory
    del dataset, train_dataset, test_dataset, train_loader, test_loader
    gc.collect()
    torch.cuda.empty_cache()
